# Create reference matrices for PCAWG signatures

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os
import sys
import itertools

In [168]:
t_1536 = itertools.product('ACGT','ACGT','[','T','>','ACG',']','ACGT','ACGT')
c_1536 = itertools.product('ACGT','ACGT','[','C','>','AGT',']','ACGT','ACGT')
context1536 = dict(zip(map(''.join, itertools.chain(t_1536, c_1536)), range(1, 1537)))
context78 = dict(zip(['AC>CA', 'AC>CG', 'AC>CT', 'AC>GA', 'AC>GG', 'AC>GT', 'AC>TA', 'AC>TG', 'AC>TT', 'AT>CA',
                      'AT>CC', 'AT>CG', 'AT>GA', 'AT>GC', 'AT>TA', 'CC>AA', 'CC>AG', 'CC>AT', 'CC>GA', 'CC>GG',
                      'CC>GT', 'CC>TA', 'CC>TG', 'CC>TT', 'CG>AT', 'CG>GC', 'CG>GT', 'CG>TA', 'CG>TC', 'CG>TT',
                      'CT>AA', 'CT>AC', 'CT>AG', 'CT>GA', 'CT>GC', 'CT>GG', 'CT>TA', 'CT>TC', 'CT>TG', 'GC>AA',
                      'GC>AG', 'GC>AT', 'GC>CA', 'GC>CG', 'GC>TA', 'TA>AT', 'TA>CG', 'TA>CT', 'TA>GC', 'TA>GG',
                      'TA>GT', 'TC>AA', 'TC>AG', 'TC>AT', 'TC>CA', 'TC>CG', 'TC>CT', 'TC>GA', 'TC>GG', 'TC>GT',
                      'TG>AA', 'TG>AC', 'TG>AT', 'TG>CA', 'TG>CC', 'TG>CT', 'TG>GA', 'TG>GC', 'TG>GT', 'TT>AA',
                      'TT>AC', 'TT>AG', 'TT>CA', 'TT>CC', 'TT>CG', 'TT>GA', 'TT>GC', 'TT>GG'], range(1, 79)))

acontext = itertools.product('A', 'CGT', 'ACGT', 'ACGT')
ccontext = itertools.product('C', 'AGT', 'ACGT', 'ACGT')
context96 = dict(zip(map(''.join, itertools.chain(acontext, ccontext)), range(1, 97)))

context83 = dict(zip(['Cdel1', 'Cdel2', 'Cdel3', 'Cdel4', 'Cdel5', 'Cdel6+',
                       'Tdel1', 'Tdel2', 'Tdel3', 'Tdel4', 'Tdel5', 'Tdel6+',
                       'Cins0', 'Cins1', 'Cins2', 'Cins3', 'Cins4', 'Cins5+',
                       'Tins0', 'Tins1', 'Tins2', 'Tins3', 'Tins4', 'Tins5+',
                       '2del1', '2del2', '2del3', '2del4', '2del5', '2del6+',
                       '3del1', '3del2', '3del3', '3del4', '3del5', '3del6+',
                       '4del1', '4del2', '4del3', '4del4', '4del5', '4del6+',
                       '5+del1', '5+del2', '5+del3', '5+del4', '5+del5', '5+del6+',
                       '2ins0', '2ins1', '2ins2', '2ins3', '2ins4', '2ins5+',
                       '3ins0', '3ins1', '3ins2', '3ins3', '3ins4', '3ins5+',
                       '4ins0', '4ins1', '4ins2', '4ins3', '4ins4', '4ins5+',
                       '5+ins0', '5+ins1', '5+ins2', '5+ins3', '5+ins4', '5+ins5+',
                       '2delm1', '3delm1', '3delm2', '4delm1', '4delm2', '4delm3',
                       '5+delm1', '5+delm2', '5+delm3', '5+delm4', '5+delm5+'], range(1, 84)))


In [161]:
# Define utility functions
def compl(seq: str, reverse: bool = False):
    """
    Gets the complement of a string
    Args:
        * seq: string (does not have to be base pair)
        * reverse: set to true to reverse seq
    Returns:
        * complement of seq
    """
    return ''.join([COMPL[x] if x in COMPL.keys() else x for x in (reversed(seq) if reverse else seq)])

COMPL = {"A":"T","T":"A","G":"C","C":"G"}

def get96_from_1536(W1536):
    """
    Convert 1536 W matrix to 96 context W matrix to extract COSMIC-compatible signatures
    ________________________
    Args:
        * W1536: 1536 context W matrix
    Returns:
        * pd.Dataframe of 96 context matrix
    """
    # Initialize 96 context matrix
    context96_df = pd.DataFrame(0, index=context96 , columns=W1536.columns)
    
    # Subset for 1536 context in original df
    W1536 = W1536[W1536.index.isin(context1536)]
    
    # Define conversion function based on format
    if ">" in W1536.index[0]:
        def convert(x):
            sbs = x[3] + x[5] + x[1] + x[7]
            if sbs[0] not in ['C','A']:
                sbs = compl(sbs[:2]) + compl(sbs[-1]) + compl(sbs[-2])
            return sbs
    else:
        def convert(x):
            sbs = x[:2] + x[3:5]
            if sbs[0] not in ['C','A']:
                sbs = compl(sbs[:2]) + compl(sbs[-1]) + compl(sbs[-2])
            return sbs
        
    # For each context in 96 SNV, sum all corresponding 1536 context rows
    for context in context96:
        context96_df.loc[context] = W1536[W1536.index.map(convert) == context].astype(np.float64).sum()
    return context96_df

def _map_id_sigs(
    df: pd.DataFrame,
    ) -> pd.Series:
    """
    Map Insertion-Deletion Substitution Signatures.
    -----------------------
    Args:
        * df: pandas.core.frame.DataFrame with index to be mapped
    Returns:
        * pandas.core.series.Series with matching indices to input cosmic
    """
    def _convert_to_cosmic(x):
        i1 = 'DEL' if 'del' in x else 'INS'
        if x[0].isdigit():
            i2 = 'MH' if 'm' in x else 'repeats'
            i3 = re.search('[\d+]+', x).group()
        else:
            i2 = x[0]
            i3 = '1'
        i4 = re.search('[\d+]+$', x).group()
        if i1 == 'DEL' and i2 != 'MH':
            i4 = str(int(i4[0]) - 1) + i4[1:]
        return '_'.join([i1, i2, i3, i4])

    if df.index.name is None: df.index.name = 'index'
    df_idx = df.index.name

    context_s = df.reset_index()[df_idx]
    return context_s.apply(_convert_to_cosmic)


In [4]:
# Define paths
REF_PATH = "BI_COMPOSITE_SNV_DNP_INDEL.signature.012718.unnormalized.txt"

In [176]:
# Read PCAWG signatures
ref_df = pd.read_csv(REF_PATH, sep='\t')

# Read COSMIC signatures
cosmic_sbs_df = pd.read_csv("../cosmic_v3/sa_cosmic3_sbs.tsv", sep='\t', index_col=0)
cosmic_dbs_df = pd.read_csv("../cosmic_v3/sa_cosmic3_dbs.tsv", sep='\t', index_col=0)
cosmic_id_df = pd.read_csv("../cosmic_v3/sa_cosmic3_id.tsv", sep='\t', index_col=0)


In [226]:
# Annotate Substitution Type
ref_df['Substitution Type'] = ref_df['feature'].map(lambda x: x[:3] if (x[1] == '>') else (x[:5] if '>' in x else '-'))
ref_df['Somatic Mutation Type'] = ref_df.iloc[:1536].loc[:,"feature"].map(lambda x: x[7:9] + '[' + x[:3] + ']' + x[10:]
                                                                         ).append(ref_df.iloc[1536:1536+78].loc[:,"feature"]
                                                                                 ).append(ref_df.iloc[1536+78:].loc[:,"feature"])

# Remove BI_COMPOSITE_ prefix
ref_df.columns = ref_df.columns.map(lambda x: x[x.index("SBS"):] if x.startswith('BI_COMPOSITE') else x)
ref_df.iloc[1536:1536+78].loc[:,'Somatic Mutation Type'] = ref_df.iloc[1536:1536+78]['Somatic Mutation Type'].map(
    lambda x: compl(x[1] + x[0]) + '>' + compl(x[4] + x[3]) if x not in cosmic_dbs_df['Somatic Mutation Type'].to_list() else x)
ref_df.iloc[1536:1536+78].loc[:,'Substitution Type'] = ref_df.iloc[1536:1536+78]['Somatic Mutation Type']
ref_df.iloc[1536:1536+78].loc[:,'feature'] = ref_df.iloc[1536:1536+78]['Somatic Mutation Type']

# PCAWG Composite reference
ref_df = ref_df[ref_df.columns[ref_df.columns.str.startswith('SBS')].to_list() + ['Somatic Mutation Type', 'feature', 'Substitution Type']]

# Subset for 96 context trinucleotide 
ref_sbs96_df = ref_df.iloc[:1536].copy().set_index('Somatic Mutation Type')
ref_sbs96_df = get96_from_1536(ref_sbs96_df.drop(columns=["feature", "Substitution Type"]))
ref_sbs96_df = ref_sbs96_df.reset_index()
ref_sbs96_df = ref_sbs96_df.rename(columns={'index':'feature'})
ref_sbs96_df.loc[:,'Somatic Mutation Type'] = ref_sbs96_df['feature'].map(lambda x: x[2] + '[' + x[0] + '>' + x[1] + ']' + x[3])
ref_sbs96_df.loc[:,'Substitution Type'] = ref_sbs96_df['Somatic Mutation Type'].map(lambda x: x[2:5])

# PCAWG composite 96 reference
ref_composite96_df = pd.concat([ref_sbs96_df, ref_df[1536:]])
ref_composite96_df = ref_composite96_df[ref_composite96_df.columns[ref_composite96_df.columns.str.startswith('SBS')].to_list() + 
                                        ['Somatic Mutation Type', 'feature', 'Substitution Type']]
ref_composite96_df = ref_composite96_df.reset_index(drop=True)

# PCAWG 1536 SBS + ID reference
ref_1536sbs_id_df = pd.concat([ref_df.iloc[:1536], ref_df.iloc[1536+78:]])

# PCAWG 96 SBS + ID reference
ref_96sbs_id_df = pd.concat([ref_composite96_df.iloc[:96], ref_composite96_df.iloc[96+78:]])

# PCAWG 1536 reference
ref_1536sbs_df = ref_df.iloc[:1536]

# Write files to reference paths
def normalize_sigs(df):
    return df.apply(lambda x: x/x.sum() if x.name.startswith('SBS') else x, 0)


In [220]:
old_pcawg_composite_df = pd.read_csv("sa_PCAWG_composite.tsv", sep='\t')
ref_df.apply(lambda x: x/x.sum() if x.name.startswith('SBS') else x, 0).round(7).compare(old_pcawg_composite_df.round(7))

Somatic Mutation Type           feature       
                      self     other    self  other
1560                   NaN       NaN   CG>TT  CG>AA
1561                   NaN       NaN   CG>GT  CG>AC
1563                   NaN       NaN   CG>TC  CG>GA
1581                   NaN       NaN   TA>GT  TA>AC
1582                   NaN       NaN   TA>CT  TA>AG
...                    ...       ...     ...    ...
1692           DEL_MH_5+_1   5+delm1     NaN    NaN
1693           DEL_MH_5+_2   5+delm2     NaN    NaN
1694           DEL_MH_5+_3   5+delm3     NaN    NaN
1695           DEL_MH_5+_4   5+delm4     NaN    NaN
1696          DEL_MH_5+_5+  5+delm5+     NaN    NaN

[89 rows x 4 columns]

In [222]:
old_pcawg_composite96_df = pd.read_csv("sa_PCAWG_composite96.tsv", sep='\t')[ref_composite96_df.columns]
ref_composite96_df.apply(lambda x: x/x.sum() if x.name.startswith('SBS') else x, 0).round(7).compare(old_pcawg_composite96_df.round(7))

Somatic Mutation Type          
                     self     other
174             DEL_C_1_0     Cdel1
175             DEL_C_1_1     Cdel2
176             DEL_C_1_2     Cdel3
177             DEL_C_1_3     Cdel4
178             DEL_C_1_4     Cdel5
..                    ...       ...
252           DEL_MH_5+_1   5+delm1
253           DEL_MH_5+_2   5+delm2
254           DEL_MH_5+_3   5+delm3
255           DEL_MH_5+_4   5+delm4
256          DEL_MH_5+_5+  5+delm5+

[83 rows x 2 columns]